In [1]:
# your code

# Задание 1

Город разбит на квадраты и для каждого квадрата известно количество людей, живущих и работающих на территории этого квадрата. Эти данные находятся в файле `data.csv`. Например, на территории квадрата `85881` живет 101 человек и работает 28 человек.

Также город разбит на административные районы, их существенно меньше. Для каждого квадрата известно то, с какими районами он пересекается и по какой части своей площади. Эти данные находятся в файле `area2district.csv`. Например, примерно 38% площади квадрата `91422` составляет район `55`.

От Вас требуется предложить способ (и реализовать его) расчета количества людей, живущих и работающих в каждом административном районе соответственно. Можно считать, что внутри каждого квадрата люди распределены равномерно.

Требования:
   * В каждом районе должно жить и работать целое неотрицательное количество людей
   * Сумма проживающих и работающих жителей города не должна измениться

В данной задаче нет строго критерия оценивания, но важно сделать упор на скорость работы, в частности не рекомендуется пользоваться циклами `for`. 

In [2]:
import pandas as pd
import numpy as np
from iteround import saferound

df_cell2adm = pd.read_csv('area2district.csv', sep=',')
df_data = pd.read_csv('data.csv', sep=',')

In [3]:
display(df_cell2adm.head())
display(df_cell2adm.shape)
display(df_data.head())
display(df_data.shape)

,areaid,districtid,percent
0,89012,478,1.000000
1,123048,55,0.984496
2,34536,7,1.000000
3,91422,55,0.380014
4,106142,55,0.912069


(217181, 3)

,areaid,home,job
0,144639,1,1
1,146472,1,1
2,18242,2,1
3,85881,101,28
4,27667,136,48


(146296, 3)

In [4]:
#смерджим таблицы по id

df_merged = df_cell2adm.merge(df_data, on='areaid', how='outer')

In [5]:
df_merged['district_job'] = df_merged['job'] * df_merged['percent']
df_merged['district_home'] = df_merged['home'] * df_merged['percent']

In [6]:
df_people = (
    df_merged
    .groupby('districtid')
    .agg(home = ('district_home', sum), job = ('district_job', sum))
    .reset_index(drop=True)
)

df_people.head()

,home,job
0,32625.162519,13566.430665
1,3716.605467,535.239616
2,46330.593462,18062.574715
3,3966.771421,1243.848413
4,4833.746494,391.347340


Теперь нужно побороть дробных людей)) Придумал для этого два варианта

**Вариант 1 (рандомно-традиционный)**

In [7]:
print('Разница сумм до округления и округленных:\n', df_people.round().sum() - df_people.sum())

Разница сумм до округления и округленных:
 home    13.0
job      2.0
dtype: float64


In [8]:
sum_home_float = df_people.home.sum()
df_people.home = df_people.home.astype('int')
sum_home_int = df_people.home.sum()
ones_num = int(sum_home_float - sum_home_int)
df_people.iloc[:ones_num, 0] += 1

In [9]:
sum_job_float = df_people.job.sum()
df_people.job = df_people.job.astype('int')
sum_job_int = df_people.job.sum()
ones_num = int(sum_job_float - sum_job_int)
df_people.iloc[:ones_num, 1] += 1

In [22]:
if df_people.sum()[0] == df_data.sum()[1] and df_people.sum()[1] == df_data.sum()[2]:
    print('Дробные люди побороты! Сумма жителей не изменилась.')
else:
    print('Боремся дальше!')

Дробные люди побороты! Сумма жителей не изменилась.


In [11]:
df_people.tail()

,home,job
438,5557,1934
439,53653,19837
440,125351,76267
441,12271,4146
442,6073,1853


**Вариант 2 (кастомно-библиотечный)**

Вернем дробные значения и заново их округлим

In [12]:
df_merged['district_job'] = df_merged['job'] * df_merged['percent']
df_merged['district_home'] = df_merged['home'] * df_merged['percent']

In [13]:
df_people = (
    df_merged
    .groupby('districtid')
    .agg(home = ('district_home', sum), job = ('district_job', sum))
    .reset_index(drop=True)
)

df_people.head()

,home,job
0,32625.162519,13566.430665
1,3716.605467,535.239616
2,46330.593462,18062.574715
3,3966.771421,1243.848413
4,4833.746494,391.347340


In [14]:
home_list = list(df_people.home)
job_list = list(df_people.job)

In [15]:
home_list_rounded = saferound(home_list, 0)
job_list_rounded = saferound(job_list, 0)

In [16]:
data = {'home': home_list_rounded, 'job': job_list_rounded}
df_people_rounded = pd.DataFrame(data=data)
df_people_rounded

,home,job
0,32625.0,13566.0
1,3717.0,535.0
2,46331.0,18063.0
3,3967.0,1244.0
4,4834.0,391.0
...,...,...
438,5557.0,1934.0
439,53654.0,19838.0
440,125351.0,76268.0
441,12272.0,4147.0


In [17]:
df_people_rounded.sum()

home    19160892.0
job      9111365.0
dtype: float64

In [18]:
df_data.sum()

areaid    13901796637
home         19160892
job           9111365
dtype: int64

In [19]:
df_people_rounded = df_people_rounded.astype('int')

In [23]:
if df_people_rounded.sum()[0] == df_data.sum()[1] and df_people_rounded.sum()[1] == df_data.sum()[2]:
    print('Дробные люди вновь побороты! Сумма жителей не изменилась')
else:
    print('Боремся дальше!')

Дробные люди вновь побороты! Сумма жителей не изменилась


In [21]:
df_people_rounded.tail()

,home,job
438,5557,1934
439,53654,19838
440,125351,76268
441,12272,4147
442,6073,1854
